In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import quote
from tqdm import tqdm
import requests
import math
from selenium.webdriver.common.keys import Keys
import pandas as pd 

In [2]:
base_url = f'https://www.a-ha.io'
main_url = f'https://www.a-ha.io/search/%EB%B6%80%EB%8F%99%EC%82%B0%20%ED%8C%81'
driver = webdriver.Chrome()
driver.get(main_url)

In [3]:
# 파업창 닫기
driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[1]/div[10]/div[1]/div/div/div/div[4]/div[2]/button').click()

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = soup.select('.box.tw-bg-transparent.min-height > div')
len(divs)
div = divs[0]
sub_href = div.select_one('.a-card__header > a')['href']
sub_href

'/questions/4533cd415d92bf728148a070ae17e366?status=waiting,ready,outdated&order=recent&aha_term=%EB%B6%80%EB%8F%99%EC%82%B0%20%ED%8C%81'

In [5]:
data = []
for div in divs:
    sub_href = div.select_one('.a-card__header > a')['href']
    sub_url = base_url + sub_href
    sub_res = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(sub_url)
    time.sleep(2)
    
    sub_soup2 = BeautifulSoup(driver.page_source, 'html.parser')
    qs = sub_soup2.select('.a-card.card-question > .a-card__main > .a-card__content > .editor__content > p')
    question = ''
    for q in qs:
        text = q.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        question += text
    
    
    ans = sub_soup2.select('.a-card.cardAnswer')[0]
    ans2 = ans.select('.a-card__main > .a-card__content > .editor__content > p')
    answer = ''
    for a in ans2:
        text = a.get_text().strip()
        if text == '\u200b' or text == '':
            continue
        answer += text
        
    driver.close()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
    
    data.append({'구분': '팁', '질문': question, '답변': answer})

driver.close()
    

TypeError: 'NoneType' object is not subscriptable

In [6]:
df = pd.DataFrame(data)
df

,구분,질문,답변
0,팁,아파트를 매도할때 집 앞에 부동산에만 얘기를 할지동네 부동산에 전부다 얘기를 할지 ...,안녕하세요. 한결같은고양이32입니다.최소3군데가봐야정확한가격이나와요부동산중계소마다조...
1,팁,제꿈이 부동산을 운영하면서 매매도 열심히해드리고 돈도 많이벌고싶은 22살입니다 부동...,안녕하세요. 박어상 공인중개사입니다.부동산을 운영하고 돈을 버는 것은 나이와는 전혀...
2,팁,사정이 좋지않아서 지금 매수세가 없지만 집을 팔아야하네요ㅜㅜ근처부동산에는 전화돌려보...,안녕하세요. 홍성택 공인중개사입니다.지금 부동산시장이 얼어붙었죠.다들 관망중인듯요....
3,팁,곧 계약이 다되가서 월세 방을 구해야하는데요이 집 구할때도 부동산 사람이 의도적으로...,안녕하세요. 서주환 공인중개사입니다.여러군데 공인중개사를 통해 다양한 매물을 확인 ...
4,팁,반려견이 2마리있는데 부동산을 하루에 15곳도 가봤지만하나같이 집이 없다 또는 집주...,안녕하세요. 양정섭 공인중개사입니다.안타깝지만 집주인이 반려견을 허락해야지 계약이 ...
5,팁,월세살면서 월세나가는게 아까워 전세자금을 마련하기위해 저축을 차곡차곡 쌓고 있는데요...,안녕하세요. 주영민 공인중개사입니다.요즘 이런 질문을 많이 받는 것 보면 다들 걱정...
6,팁,부동산 경매를 시작할려고 하는 회사원입니다 시작할려고하니 두려움이 눈앞을 가립니다~...,"안녕하세요. ""부동산코칭 박샘"" 박진혁 공인중개사입니다.부동산경매를 잘 할수 있는 ..."
7,팁,제가 이번에 부동산 자격증 시험을 준비한다고 교재랑 모의고사 문제집을 사려니깐 가격...,안녕하세요. 너그러운코뿔소140입니다.인터넷 서점 에서 구매할 때 중고를 살 수 있...
8,팁,종합소득세가 너무 많이 나오는데요 절세하는 팁좀 알려주세요연매출 4억정도 되는 회산...,안녕하세요? 아하(Aha) 세무·회계 분야 전문가 문용현세무사입니다. 질문하신 내용...
9,팁,안녕하세요.이번에 부동산 매수하여 곧 잔금 예정인 사람입니다. 은행 대출이 있어서 ...,안녕하세요. 강한나팔새38입니다.어플을 이용해서 서비스를 받는 것 더 나아보입니다....


In [7]:
df.to_csv('data/팁309_3.csv')